In [0]:
databricks bundle validate


spark

In [0]:

from pyspark.sql.types import *
from pyspark.sql.functions import *
import os 
import sys 
project_path = os.path.join(os.getcwd(),'..','..')
sys.path.append(project_path)
from utils.transformation import reusable

In [0]:
project_path

'/Workspace/Users/shubhamsaw56783@outlook.com/spotify_dab/src/Sliver/../..'

###AutoLoader


In [0]:
df_user = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimUser/checkpoints")\
    .load("abfss://bronze@spotifystoragesaw.dfs.core.windows.net/DimUser") 
      
 

In [0]:
df_user_upper = df_user.withColumn("user_name",upper(col("user_name")))
# display(df_user_upper)


In [0]:
df_user_obj =  reusable()
df_user_drop = df_user_obj.dropColumn(df_user,["_rescued_data"])
df_user_duplicate = df_user.dropDuplicates(["user_id"])
# display(df_user_drop)
# display(df_user_duplicate)



In [0]:
df_user.writeStream.format('delta')\
        .outputMode("append")\
        .option("checkpointLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimUser/checkpoints")\
        .trigger(once = True)\
        .option("path","abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimUser/data") \
        .toTable("spotify_cata.sliver.DimUser") 

         

##DimArtish


In [0]:
df_art = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimArt/checkpoints")\
    .option("schemaEvolutionMode", "addNewColumns")\
    .load("abfss://bronze@spotifystoragesaw.dfs.core.windows.net/DimArtist")  
# display(df_art)   

In [0]:
df_art_obj = reusable()

df_art = df_art_obj.dropColumn(df_art,["_rescued_data"])
df_art_duplicate = df_art.dropDuplicates(["artist_id"])
# display(df_art)

In [0]:
df_art.writeStream.format('delta')\
        .outputMode("append")\
        .option("checkpointLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimArt/checkpoints")\
        .trigger(once = True)\
        .option("path","abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimArt/data")\
        .toTable("spotify_cata.sliver.DimArt")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:720)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:440)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

##DimTrack




In [0]:
df_track = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimTra/checkpoints")\
    .option("schemaEvolutionMode", "addNewColumns")\
    .load("abfss://bronze@spotifystoragesaw.dfs.core.windows.net/DimTrack") 



In [0]:

df_art_duplicate = df_art.dropDuplicates(["artist_id"])


In [0]:
df_track.writeStream.format('delta')\
        .outputMode("append")\
        .option("checkpointLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimTrack/checkpoints")\
        .trigger(once = True)\
        .option("path","abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimTrack/data")\
        .toTable("spotify_cata.sliver.DimTrack")

In [0]:
df_track = df_track.withColumn("durationFlag",when(col("duration_sec")<150,"low")\
                                              .when(col("duration_sec")<300,"medium")\
                                              .otherwise("High"))    
                                               
df_track = df_track.withColumn("track_name",regexp_replace(col("track_name"),'-',' '))   

df_track_obj = reusable()

df_track = df_track_obj.dropColumn(df_track,["_rescued_data"])

# display(df_track)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:720)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:440)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

##DimDate



In [0]:
df_date = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimDate/checkpoints")\
    .option("schemaEvolutionMode", "addNewColumns")\
    .load("abfss://bronze@spotifystoragesaw.dfs.core.windows.net/DimDate")
display(df_date)

In [0]:
df_date = reusable().dropColumn(df_date,["_rescued_data"])

In [0]:
df_date.writeStream.format('delta')\
        .outputMode("append")\
        .option("checkpointLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimDate/checkpoints")\
        .trigger(once = True)\
        .option("path","abfss://sliver@spotifystoragesaw.dfs.core.windows.net/DimDate/data")\
        .toTable("spotify_cata.sliver.DimDate")

##FactStream


In [0]:


df_fact.writeStream.format('delta')\
        .outputMode("append")\
        .option("checkpointLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/FactStream/checkpoints")\
        .trigger(once = True)\
        .option("path","abfss://sliver@spotifystoragesaw.dfs.core.windows.net/FactStream/data")\
        .toTable("spotify_cata.sliver.FactStream")
   

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://sliver@spotifystoragesaw.dfs.core.windows.net/FactStream/checkpoints")\
    .option("schemaEvolutionMode", "addNewColumns")\
    .load("abfss://bronze@spotifystoragesaw.dfs.core.windows.net/FactStream")

In [0]:
df_fact = reusable().dropColumn(df_fact,["_rescued_data"])

In [0]:
display(df_fact)   